In this first part I will reduce the data sizes by convert the values. For this is use the work share by Chris DEOTTE (https://www.kaggle.com/competitions/amex-default-prediction/discussion/328054), the reust is i convert the files 50Go to 3.5 GO in parquet format.

In [1]:
!pip install pandas

In [2]:
import pandas as pd

In [3]:
print('code start')
data = pd.read_csv('train_data.csv', parse_dates=['S_2'])
labels = pd.read_csv('train_labels.csv')
print('data loading on gpu')

code start
data loading on gpu


In [4]:
df = pd.merge(data, labels, on='customer_ID')

In [5]:
# Reduce the size of the Customer ID by keeping only the last 16 digits
df['code'] = df['customer_ID'].apply(lambda x: int(x[-16:], 16)).astype('int64')

# Create a dictionary of the customer ID for the final submission
MAPPING = df['customer_ID'].to_dict()
print('Data encoding done')

Data encoding done


In [6]:
# Convert 'code' to int32 to reduce more the data set size
df['code'] = df['code'].astype('int32')

# Set 'code' as index and drop 'customer_ID'
df.set_index('code', inplace=True)
df.drop('customer_ID', axis=1, inplace=True)
print('Index set on code')

Index set on code


In [7]:
# Function to check if a series contains only integer-like values
def is_integer_like(series):
    series_no_na = series.dropna()
    return (series_no_na == series_no_na.astype(int)).all()


# Identify float columns that contain only integer-like values
float_columns = df.select_dtypes(include=['float64']).columns
integer_like_columns = [col for col in float_columns if is_integer_like(df[col])]

print("Columns that are float but contain only integer-like values:")
print(integer_like_columns)

Columns that are float but contain only integer-like values:
['D_66', 'D_68', 'B_30', 'D_87', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126']


In [8]:
# Change integer-like float columns to int8 to reduce size
for col in integer_like_columns:
    df[col] = df[col].fillna(0).astype('int8')

# Convert remaining float columns to float32
for col in float_columns:
    df[col] = df[col].astype('float32')

print('Float conversion done')

Float conversion done


In [9]:
# Ensure no data is lost: no duplicates, then save the DataFrame in parquet and feather formats
duplicate_counts_reduction = df.index.duplicated(keep=False).sum()
print(f'Duplicate reduction count: {duplicate_counts_reduction}')

Duplicate reduction count: 5526332


In [11]:
df.to_parquet('data_reduce.parquet')
print('Data saved in Parquet')

Data saved in Parquet


In [12]:
df.to_feather('data_reduce.feather')
print('Data saved in Feather')

Data saved in Feather
